### Analyzing public and hybrid cloud networks

XXXX

In [1]:
# Import packages and load questions
%run startup.py

load_questions()

### Initializing the Network and Snapshot

SNAPSHOT_PATH below can be updated to point to a custom snapshot directory, see the Batfish instructions for how to package data for analysis. More example networks are available in the networks folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "hybrid-cloud"
SNAPSHOT_NAME = "snapshot"

SNAPSHOT_PATH = "networks/hybrid-cloud"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/hybrid-cloud).

The AWS portion of the configuration is in the aws_configs subfolder. These are JSON files obtained via AWS APIs. An example script that packages AWS data into a Batfish snapshot is [here](https://github.com/ratulm/bf-aws-snapshot).

![example-bgp-network](https://raw.githubusercontent.com/batfish/pybatfish/aws-notebook/jupyter_notebooks/networks/hybrid-cloud/hybrid-cloud.png)